## Tools

Please skip this code and go to [Set Up](#Set-Up).

In [1]:
from array import array
from app import Application
from blocks import transform, acm, rt, bbs
from time import time

import math as mt
import numpy as np
import pandas as pd
import os


def time_it(fungsi, masukan):
    checkpoint = time()
    
    keluaran = fungsi(masukan)
    
    waktu = time() - checkpoint
    return waktu, keluaran

def encrypt(kunci, plainfile):
    data = {}

    # TRANSFORMATION
    data[0], plainbytes = time_it(transform.encryption_formatting, plainfile)
    data[1], plainbytes = time_it(transform.pad_bytes, plainbytes)

    # RANDOMIZE TEXT
    data[2], cipherbytes = time_it(rt.RT(bbs.BBS(kunci[0], kunci[1], kunci[2])).encrypt, plainbytes)

    # TRANSFORMATION
    data[3], ciphermatrix = time_it(transform.bytes_to_matrix, cipherbytes)
    data[4], plainimage = time_it(transform.matrix_to_image, ciphermatrix)

    # ARNOLD'S CAT MAP
    data[5], cipherimage = time_it(acm.ACM(kunci[3], kunci[4], kunci[5]).encrypt, plainimage)
    
    data[6] = sum(data.values())
    
    return data, cipherimage

def decrypt(kunci, cipherimage):
    data = {}

    # ARNOLD'S CAT MAP
    data[0], plainimage = time_it(acm.ACM(kunci[3], kunci[4], kunci[5]).decrypt, cipherimage)

    # TRANSFORMATION
    data[1], ciphermatrix = time_it(transform.image_to_matrix, plainimage)
    data[2], cipherbytes = time_it(transform.matrix_to_bytes, ciphermatrix)

    # RANDOMIZE TEXT
    data[3], plainbytes = time_it(rt.RT(bbs.BBS(kunci[0], kunci[1], kunci[2])).decrypt, cipherbytes)

    # TRANSFORMATION
    data[4], plainbytes = time_it(transform.strip_bytes, plainbytes)
    data[5], plainfile = time_it(transform.decryption_formatting, plainbytes)
    
    data[6] = sum(data.values())
    
    return data, plainimage


# PUT THIS IF YOU WORKING WITH PRIMES
primes = array('I')

a = open('primes/100000.txt', 'r')
lines = a.readlines()

for line in lines[5:]:
    for prime in line[:-2].split():
        prime = int(prime)
        if prime % 4 == 3 and prime < 65536:
            primes.append(int(prime))
# END HERE

def generate_s(p, q):
    m = p * q
    s = np.random.randint(1, m)
    while mt.gcd(s, m) != 1:
        s = np.random.randint(1, m)
    return s

# Set Up

Please define following constant and run the block [Run](#Run)

### Input File

- The average `B = 321000` is [the average size of an office document](https://blogs.technet.microsoft.com/dangl/2012/10/18/what-is-the-average-size-of-an-office-document/).
- The maximum value of parameter `a` and `b` for `B = 3210001` is `463` considering the requirement for `a` and `b` are both positive integers and `a <= N` and `b <= N`, `N` is the maps dimension. The number 463 comes from our investigation using formula `N = ceil(sqrt(ceil((B * 2) / 3)))`.
- The number `30211`, `627073506`
- The maximum value of parameter `n` is setted to `300`. This is to limit our computing performance.
- We had found a dataset of two bytes prime number. From that data, we had done some filter to compromise our prime are `3 (mod 4)`. And the maximum prime number we got is `65519`, the  minimum is `3`, and the number of our primes is `3284` numbers.

In [2]:
#DATA TEST is B, p, q, s, a, b, n

DATA_TEST = []
# ACM DISCRETE over n
for n in np.linspace(10, 300, 30):
    data = [321000, 30203, 30211, 627073506, 1, 1, mt.ceil(n)]
    DATA_TEST.append(data)
        
# ACM GENERAL EQUAL over n
for n in np.linspace(10, 300, 30):
    # b = a = 463 // 2
    data = [321000, 30203, 30211, 627073506, 231, 231, mt.ceil(n)]
    DATA_TEST.append(data)
        
# ACM GENERAL ANY over n
for n in np.linspace(10, 300, 30):
    # a and b are a couple of true random number (set by user) on [1, 463]
    data = [321000, 30203, 30211, 627073506, 231, 132, mt.ceil(n)]
    DATA_TEST.append(data)
    
# ACM GENERAL EQUAL over a
for a in np.linspace(30203, 463, 30):
    a = mt.ceil(a)
    data = [321000, 30203, 30211, 627073506, a, a, 10]
    DATA_TEST.append(data)
    
# RT over p and q using random s
for i in np.linspace(1, len(primes)-1, 30):
    i = mt.ceil(i)
    p = primes[i]
    q = primes[i - 1]
    s = generate_s(p, q)
    data = [321000, p, q, s, a, a, 10]
    DATA_TEST.append(data)
    
# RT over p and q using random s
for i in np.linspace(1, len(primes)-1, 30):
    i = mt.ceil(i)
    # This experiment is to show the correlation between the value of `m` and time required.
    # Since we are not allowed to use mutual `p` and `q`, hence we are using neighborhood `p` and `q`
    # to demonstrate inclining `m = p * q`.
    p = primes[i]
    q = primes[i - 1]
    s = generate_s(p, q)
    data = [321000, p, q, s, 231, 132, 150]
    DATA_TEST.append(data)

# over B
# we have data of common office size, our B data should refer to that
for B in np.linspace(0, 1000000, 31):
    if B == 0:
        continue
    B = mt.ceil(B)
    data = [B, 30203, 30211, 627073506, 231, 132, 150]
    DATA_TEST.append(data)
    

In [3]:
enc = []
dec = []
# for TEST in [[321000, 30203, 30211, 627073506, 1, 1, 2], ]:
for TEST in DATA_TEST:
    print(TEST)
    plainfile = (170).to_bytes(1, 'little') * TEST[0]
    
    print('is encrypting . . .')
    data_encrypt, cipherimage = encrypt(TEST[1:], plainfile)

    print('is decrypting . . .')
    data_decrypt, plainfile = decrypt(TEST[1:], cipherimage)
    
    enc.append(data_encrypt)
    dec.append(data_decrypt)

df_enc = pd.DataFrame(enc)
df_dec = pd.DataFrame(dec)
df_dtt = pd.DataFrame(DATA_TEST)

df_enc.to_csv("time_enc.csv", index=False)
df_dec.to_csv("time_dec.csv", index=False)
df_dtt.to_csv("time_dtt.csv", index=False)
print('Done')

[321000, 30203, 30211, 627073506, 1, 1, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 20]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 30]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 40]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 50]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 60]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 70]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 80]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 90]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 100]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 110]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 1, 1, 120]
is encrypting . . .
is decryp

is decrypting . . .
[321000, 30203, 30211, 627073506, 26101, 26101, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 25076, 25076, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 24050, 24050, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 23025, 23025, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 21999, 21999, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 20974, 20974, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 19948, 19948, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 18923, 18923, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 17897, 17897, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 16872, 16872, 10]
is encrypting . . .
is decrypting . . .
[321000, 30203, 30211, 627073506, 15846, 15846, 10]
is encry

is decrypting . . .
[266667, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[300000, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[333334, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[366667, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[400000, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[433334, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[466667, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[500001, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[533334, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[566667, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[600000, 30203, 30211, 627073506, 231, 132, 150]
is encrypting . . .
is decrypting . . .
[